# Loading libraries and data

In [4]:
import pandas as pd
from datetime import datetime

In [5]:
df_train = pd.read_csv('data/event-log-training.csv')

In [6]:
df_train

,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp
0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000
1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000
2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000
3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000
4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000
...,...,...,...,...,...,...,...,...
36623,42949673754,18,Simulated process instance,Print,PRN1,Pressure Roller Spin Stop,complete,25-12-1970 15:51:00.000
36624,42949673755,18,Simulated process instance,Print,PRN1,Fusing,complete,25-12-1970 16:05:00.000
36625,42949673756,18,Simulated process instance,Print,PRN1,Wipe Toner on Drum,complete,25-12-1970 16:10:00.000
36626,42949673757,18,Simulated process instance,Print,PRN1,Erase Charge on Drum,complete,25-12-1970 16:22:00.000


# Convert timestamp to datetime

In [7]:
date_list = []

for time in df_train['event time:timestamp']:
    datex = time[:-4]
    date = datetime.strptime(datex, '%d-%m-%Y %H:%M:%S')

    date_list.append(date)

In [8]:
df_train['time and date'] = date_list

In [9]:
df_train

,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,time and date
0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000,1970-01-01 01:00:00
1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000,1970-01-01 01:15:00
2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000,1970-01-01 01:26:00
3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000,1970-01-01 01:38:00
4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000,1970-01-01 01:51:00
...,...,...,...,...,...,...,...,...,...
36623,42949673754,18,Simulated process instance,Print,PRN1,Pressure Roller Spin Stop,complete,25-12-1970 15:51:00.000,1970-12-25 15:51:00
36624,42949673755,18,Simulated process instance,Print,PRN1,Fusing,complete,25-12-1970 16:05:00.000,1970-12-25 16:05:00
36625,42949673756,18,Simulated process instance,Print,PRN1,Wipe Toner on Drum,complete,25-12-1970 16:10:00.000,1970-12-25 16:10:00
36626,42949673757,18,Simulated process instance,Print,PRN1,Erase Charge on Drum,complete,25-12-1970 16:22:00.000,1970-12-25 16:22:00


# Add in true next event and time it takes for case 4

In [10]:
df_train['event lifecycle:transition'].value_counts()

complete    33802
start        2826
Name: event lifecycle:transition, dtype: int64

In [11]:
dct_events = dict(df_train['event concept:name'].value_counts())

In [12]:
df_train['case concept:name'].value_counts()

12    2140
33    1594
42    1390
24    1285
6     1198
      ... 
27      62
80      59
56      55
4       43
51      43
Name: case concept:name, Length: 80, dtype: int64

In [13]:
df_train.groupby(by = ['case concept:name', 'time and date'], as_index = False)
df_case4 = df_train[df_train['case concept:name'] == 4]

In [14]:
#df_case4

Add event column

In [15]:
event_lst4 = [event for event in df_case4['event concept:name']]

event_lst4 = event_lst4[1:]
event_lst4.append('-')
    

In [16]:
df_case4['next_event'] = event_lst4

C:\Users\20204502\AppData\Local\Temp/ipykernel_16144/3880512715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case4['next_event'] = event_lst4


Add time to next event column

In [17]:
nexttime_lst1 = [time for time in df_case4['time and date']]

nexttime_lst = nexttime_lst1[1:]
nexttime_lst.append(nexttime_lst[-1])

In [18]:
time_diff = []

for i in range(len(nexttime_lst)):
    time_diff.append(nexttime_lst[i] - nexttime_lst1[i])

In [19]:
df_case4['time to next event'] = time_diff

C:\Users\20204502\AppData\Local\Temp/ipykernel_16144/1624469924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case4['time to next event'] = time_diff


In [20]:
df_case4.head()

,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,time and date,next_event,time to next event
64,146028888064,4,Simulated process instance,Copy/Scan,PRN1,Job,start,01-01-1970 11:23:00.000,1970-01-01 11:23:00,Copy/Scan,271 days 01:10:00
26983,146028888065,4,Simulated process instance,Copy/Scan,PRN1,Copy/Scan,complete,29-09-1970 12:33:00.000,1970-09-29 12:33:00,Collect Copy/Scan Options,0 days 00:18:00
26984,146028888066,4,Simulated process instance,Copy/Scan,PRN1,Collect Copy/Scan Options,complete,29-09-1970 12:51:00.000,1970-09-29 12:51:00,Place Doc,0 days 00:08:00
26985,146028888067,4,Simulated process instance,Copy/Scan,PRN1,Place Doc,complete,29-09-1970 12:59:00.000,1970-09-29 12:59:00,Illuminate Document,0 days 00:19:00
26986,146028888068,4,Simulated process instance,Copy/Scan,PRN1,Illuminate Document,complete,29-09-1970 13:18:00.000,1970-09-29 13:18:00,Move Scan Head,0 days 00:22:00


# Next event and time column for all data


In [21]:
cases = list(df_train['case concept:name'].unique())

In [19]:
df_result = pd.DataFrame()

In [29]:
int_dict = {}
for case in cases:
    df_case = df_train[df_train['case concept:name'] == case].reset_index()
    for index, row in df_case.iterrows():
        if row['event concept:name'] in int_dict:
            if index in int_dict[row['event concept:name']]:
                int_dict[row['event concept:name']][index] += 1
            else:
                int_dict[row['event concept:name']].update({index: 1})
        else:
            int_dict[row['event concept:name']] = {index: 1}

In [30]:
int_dict

{'Job': {0: 80,
  777: 1,
  112: 1,
  357: 2,
  42: 2,
  155: 1,
  1197: 1,
  119: 2,
  1177: 1,
  992: 2,
  1011: 1,
  2139: 1,
  670: 1,
  124: 1,
  144: 1,
  96: 3,
  798: 1,
  131: 1,
  158: 1,
  468: 1,
  82: 2,
  1284: 1,
  621: 1,
  65: 1,
  61: 1,
  592: 1,
  123: 2,
  1027: 1,
  1086: 1,
  1593: 1,
  658: 1,
  467: 1,
  76: 1,
  204: 1,
  786: 1,
  366: 1,
  68: 2,
  521: 1,
  1389: 1,
  876: 1,
  450: 1,
  195: 1,
  107: 1,
  1089: 1,
  117: 1,
  983: 1,
  133: 2,
  135: 1,
  146: 1,
  54: 1,
  200: 1,
  663: 1,
  563: 1,
  288: 1,
  140: 1,
  734: 1,
  612: 1,
  834: 1,
  726: 1,
  505: 1,
  153: 1,
  128: 1,
  1069: 1,
  109: 1,
  387: 1,
  102: 1,
  89: 1,
  431: 1,
  551: 1,
  58: 1,
  77: 1},
 'Remote Print': {1: 44},
 'Read Print Options': {2: 44},
 'Rasterization': {3: 44,
  66: 3,
  57: 1,
  102: 6,
  120: 4,
  67: 2,
  139: 1,
  129: 3,
  58: 3,
  84: 2,
  111: 2,
  75: 1,
  85: 1,
  112: 3,
  31: 2,
  138: 2,
  94: 2,
  30: 3,
  22: 1,
  121: 1,
  103: 1},
 'Interpr

In [ ]:
#create next time list
    nexttime_lst1 = [time for time in df_case['time and date']]
    nexttime_lst = nexttime_lst1[1:]
    nexttime_lst.append(nexttime_lst[-1])
    
    #create difference list
    time_diff = []
    for i in range(len(nexttime_lst)):
        time_diff.append(nexttime_lst[i] - nexttime_lst1[i])

In [105]:
#2140
position_dict = {}

for i in range(2140):
    init = 0
    task = 0
    for key in int_dict.keys():
        try:
            new = int_dict[key][i]
        except:
            new = 0
        if new > init:
            init = new
            task = key

    position_dict.update({i: task})

In [110]:
pred_list = [position_dict[i] for i in position_dict.keys()]


In [111]:
pred_list

['Job',
 'Remote Print',
 'Read Print Options',
 'Rasterization',
 'Interpretation',
 'Move Scan Head',
 'Interpretation',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Collect Image',
 'Illuminate Document',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Illuminate Document',
 'Accumulate Images',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Collect Image',
 'Illuminate Document',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Illuminate Document',
 'Collect Image',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Illuminate Document',
 'Collect Image',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Collect Image',
 'Illuminate Document',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filter

In [120]:
df_result = pd.DataFrame()

for case in cases: 
    
    df_case = df_train[df_train['case concept:name'] == case].copy().reset_index()

    pred_list = [position_dict[i] for i in range(len(df_case))]
    pred_list = pred_list[1:]
    pred_list.append('-')

    

    
    #create next event list
    event_lst = [event for event in df_case['event concept:name']]
    event_lst = event_lst[1:]
    event_lst.append('-')
    
    #create next time list
    nexttime_lst1 = [time for time in df_case['time and date']]
    nexttime_lst = nexttime_lst1[1:]
    nexttime_lst.append(nexttime_lst[-1])
    
    #create difference list
    time_diff = []
    for i in range(len(nexttime_lst)):
        time_diff.append(nexttime_lst[i] - nexttime_lst1[i])
    
    #append columns to df case
    df_case['Next event'] = event_lst
    df_case['Time to next event'] = time_diff


    df_case['Event predcition'] = pred_list
    
    #append df case to final df
    df_result = df_result.append(df_case)

In [ ]:
for case in cases:
    df_case = df_train[df_train['case concept:name'] == case].copy()
    df_case.reset_index(inplace=True)

In [1]:
df_result.head()

NameError: name 'df_result' is not defined